# Safe Financial AI with Claude 2 and Amazon Bedrock


## WindyCityDevFest 
## October 24, 2023

In [ ]:
import json
import re
from ai_utils import *
from data_utils import *

In [ ]:
complete("hello, to whom am I speaking?")

In [ ]:
symbols = ["SPY", "VGT", "AAPL", "MSFT", "NVDA", 
           "ADBE", "CSCO", "CRM", "ORCL", "AMD", 
           "ACN", "INTC", "TXN", "INTU", "IBM", 
           "QCOM", "MU", "PANW"]

raw_data = fetch_data(symbols, start="2023-07-03", end="2023-10-03", interval="1wk")
raw_data["Adj Close"]

In [ ]:
portfolio = {
    "AAPL": 80,
    "ACN": 20,
    "ADBE": 30,
    "AMD": 60
}

positions = pd.DataFrame(index=raw_data.index)
for (sym, shares) in portfolio.items():
    positions[sym] = raw_data["Adj Close"][sym] * shares
positions["Total"] = positions.sum(axis=1)

positions

## Does it Work? Initial Prompt and Validation

In [ ]:
prompt = f"""
Given the following time series data, consisting of investment portfolio performance data, 
as well as the portfolio total, please identify:
1. the overall trend and norm in the data, as well as
2. particular dates/rows that are outliers
3. how particular securities contributing to trends and outliers overall

Category: investment portfolio data
Overall date range: July 3, 2023 - Oct 9 2023 (Weekly)
Data:
{to_csv(positions,'${:.2f}')}
"""

print(prompt)
print("\n--waiting--\n")
print(complete(prompt)['completion'])

## Add Data, Until it Breaks

In [ ]:
prompt = f"""
Given the following time series data, consisting of securities market data for a series of individual technology stocks, 
as well as VGT, a technology-sector index fund please compare the performance of the listed securities to the index over the entire time period.
You don't have to provide an interpretation of every security provided, but please call out any exceptional high or low performers.

Category: market and reference data
Overall date range: July 3, 2023 - Oct 9 2023 (Weekly)
Data:
{to_csv(raw_data["Adj Close"],'${:.2f}')}
"""
print(prompt)
print("\n--waiting--\n")
print(complete(prompt)['completion'])

In [ ]:
def mk_gains(positions):
    percentage_change = pd.DataFrame(index=positions.index)
    for k in positions.keys():
        percentage_change[k] = 100 * (1 - positions[k].shift(1) / positions[k])
    return percentage_change.tail(-1)

performance = mk_gains(raw_data["Adj Close"])
performance

In [ ]:
prompt = f"""
Given the following time series data, consisting of investment portfolio performance data, 
as well as the portfolio total, please identify:
1. the overall trend and norm in the data, as well as
2. particular dates/rows that are outliers
3. how particular securities contributing to trends and outliers overall

Category: investment portfolio data
Overall date range: July 3, 2023 - Oct 9 2023 (Weekly)
Data:
{to_csv(performance,'{:.2f}%')}
"""

print(prompt)
print("--waiting--")
print(complete(prompt)['completion'])

## Specialized Prompts and Structured Output

In [ ]:
start_date = "2023-07-03"
end_date = "2023-10-02"
start_end = raw_data["Adj Close"].loc[[start_date, end_date]]
start_end

In [ ]:
start_end_gains = mk_gains(start_end)
start_end_gains

In [ ]:
prompt = f"""
Given the following stock performance data, including start and end prices over the time frame, 
as well as the percentage change (performance), please identify overall trends, 
as well as outliers relative to the performance of VGT, a tech index fund that is included as well.

Please tag any outlier you identify with the XML tag <outlier> for future processing, like <outlier>AMD</outlier>.

Category: market and reference data
Overall date range: July 3, 2023 - Oct 9 2023 (Weekly)
Price Data:
{to_csv(start_end,'${:.2f}')}

Performance Data:
{to_csv(start_end_gains)}
"""
print(prompt)
print("\n--waiting--\n")
tagged_results = complete(prompt)['completion']
print(tagged_results)

In [ ]:
prompt = f"""
please write me a python regular expression to extract xml tags of the form <outlier>SYMBOL</outlier>"""
print(complete(prompt)['completion'])

## Iterating over structured output

In [ ]:
outliers = re.findall(r'<outlier>(.*?)</outlier>',tagged_results)
print(outliers)

detail_results = []

for outlier in outliers:
    outlier_data = raw_data['Adj Close'][['SPY','VGT',outlier]]
    prompt = f"""
    Given the following stock performance data for {outlier}, 
    as well as reference index funds for the tech sector (VGT) and overall market (SPY),
    please relate {outlier}'s performance to overall trends, noting specific dates 
    when it overperformered or underperformed. 
    
    When you identify a date, please tag it with XML as <date>, 
    like <date>2023-07-03</date> (make sure to use YYYY-MM-DD format as well)
    
    Category: market and reference data
    Overall date range: July 3, 2023 - Oct 9 2023 (Weekly)
    Price Data:
    {to_csv(outlier_data,'${:.2f}')}    
    """
    print(prompt)
    print("\n--waiting--\n")
    result = complete(prompt)['completion']
    print(result)
    print("\n--done--\n")
    detail_results.append(result)

## Concatenate and Summarize

In [ ]:
prompt = f"""
Please summarize and contextualize the following collection of portfolio performance analysis 
from a series of preceding AI prompts.  First, I'll provide an overall summary of portfolio performance 
over the time period July 3, 2023 - Oct 9, 2023 - that overall performance analysis 
will identify several outlier securities for further analysis. Ensure that you focus on the big picture, 
while providing precise detail on particular events of interest. 
Do not provide investment advice or make any predictions - make empirical claims only.

Category: Overall performance analysis
Time Period: July 3, 2023 - Oct 9, 2023
Performance Data:
{to_csv(start_end_gains)}
Analysis:
{tagged_results}
"""

for outlier_analysis in detail_results:
    prompt = prompt + "\n\nOutlier Analysis:\n" + outlier_analysis

print(prompt)
print("\n--waiting--\n")
final_result = complete(prompt)['completion']
print(final_result)
print("\n--done--\n")